# Segmentação e criação do vetor de características

**Observação inicial**: para os trabalhos posteriores, será levado em consideração que os dados estão corretos e filtrados conforme na aula de "Pré-processamento". Portando, antes de iniciar este conteúdo, vocês terão que ter concluído as questões passadas e como **tarefa** salvar um arquivo com dados filtrados (não executar todo `preprocessing.ipynb`).

Serão utilizados nesta aula os dados sem nenhuma filtragem e com o conjunto de canais escolhidos aleatoriamente.

## Introdução

Um formato importante do *dataset* para a classificação dos dados, é estar organizado preferencialmente em duas dimensões. As linhas serão as amostras (rotuladas ou não) e as colunas, as características. Além disso, os dados para cada uma das características deve fazer algum sentido para a boa atuação do classificador. Para essa matriz final, damos o nome de `vetor de características`.

Em experimentos SSVEP-BCI, a característica mais forte é o `PSD` (*Power Spectral Density*). O `PSD`, como o nome sugere, é obtido por meio do sinal no domínio da frequência, aplicando a seguinte fórmula: $|x_i|^2$. O `PSD` potencializa a energia das frequências mais evidentes, melhorando o desempenho de classificação.

Alguns métodos da biblioteca MNE nos dão um vetor de características pronto. Porém, é interessante realizarmos algumas etapas passo a passo sem o uso inicial da biblioteca para entendermos o funcionamento do método e alterar como quisermos.


## Transformação de domínio (e segmentação)

O `shape` inicial dos dados é: `(125, 256, 1205) -> (trials, channels, data)`. Vamos aplicar a Transformada Rápida de Fourier em Tempo Curto (STFT) (após carregar e filtrar os dados):

In [8]:
import matplotlib
import mne
from scipy.signal import stft
import numpy as np

%matplotlib inline

In [9]:
# carregamento do dataset (FIF file)
epochs = mne.read_epochs('files/ssvep-epo.fif')
# filtranndo apenas alguns canais
epochs.pick_channels(['E108', 'E109', 'E116', 'E125', 'E118', 'E117', 'E126',
                      'E139', 'E127', 'E138', 'E140', 'E150', 'E151'])
print(epochs)

Reading files/ssvep-epo.fif ...
    Found the data of interest:
        t =       0.00 ...    4816.00 ms
        0 CTF compensation matrices available
125 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
<EpochsFIF  |   125 events (all good), 0 - 4.816 sec, baseline off, ~15.1 MB, data loaded,
 '1': 25
 '2': 25
 '3': 30
 '4': 25
 '5': 20>


In [10]:
# extraindo somente os dados do objeto MNE
data = epochs.get_data()
print(data.shape) # domínio do tempo

epochs_welch, freqs = mne.time_frequency.psd_welch(epochs, fmin = 4, fmax = 15) 
print(epochs_welch)

# aplicando STFT
_, _, w = stft(epochs_welch, fs=241, nperseg=11, noverlap=6)
# w = np.swapaxes(w, 3, 4)
print(w.shape)

(125, 13, 1205)
Effective window size : 1.024 (s)
[[[0.93674441 0.74141311 0.38201137 ... 0.68028774 0.5873291  0.2476712 ]
  [0.97638268 0.63740008 0.2965196  ... 0.64086363 0.59099704 0.23757344]
  [1.03881271 0.74823094 0.35052467 ... 0.76193674 0.66034644 0.25821164]
  ...
  [0.75223499 0.69451247 0.32560598 ... 0.57718996 0.40823829 0.07124297]
  [0.98549479 0.94989159 0.41630772 ... 0.65147623 0.47961005 0.1042054 ]
  [0.98260192 0.92993171 0.41861242 ... 0.61724877 0.46385364 0.0900444 ]]

 [[0.38381504 0.71549564 0.34765654 ... 0.22972511 0.45814206 0.59436434]
  [0.53593299 0.67116295 0.27774932 ... 0.13494958 0.324484   0.39715296]
  [0.55843889 0.7548622  0.30406947 ... 0.20261892 0.46116079 0.56648374]
  ...
  [0.51577375 0.45455115 0.18185857 ... 0.11417187 0.27062902 0.39060153]
  [0.46213031 0.5655062  0.35507217 ... 0.17301737 0.29036647 0.50118811]
  [0.42661521 0.51815579 0.24817847 ... 0.14329983 0.29927486 0.46798453]]

 [[0.84469972 0.73678515 0.65497281 ... 0.3498

Obemos um `shape` diferente, acrescentando uma dimensão a mais em nossos dados. Isso é devido a quantidade de janelas ou segmentos informados (`nperseg`) e a sobreposição utilizada (`overlap`). **DISCUSSÃO EM AULA**

Aplicando o `PSD` teremos:

In [11]:
W = np.abs(w) ** 2

Wabs=np.abs(w)

# w = np.reshape(w, (125, 13, 17 * 77)) # <= questão de projeto
# w = w.transpose(0, 2, 1)
# w = np.reshape(w, (125 * 1309, 13))
print(W.shape)

# shape resultante: (125, 13, 17, 77)

(125, 13, 6, 3)


## Extração de características

Não é uma boa estratégia utilizar os dados "crus" de `PSD` para a classificação. Desta forma, vamos adotar alguns algoritmos simples para reduzir uma dimensão dos dados e potencializar nossas características. Uma lista de característica é listada [por este artigo intitulado "*A wearable wireless brain-computer interface using steady-state visual evoked potentials*"](https://www.researchgate.net/publication/334854837_A_wearable_wireless_brain-computer_interface_using_steady-state_visual_evoked_potentials). Já que temos o PSD dos dados, vamos demonstrar a aplicação do "*Mean of PSD*" ou `FMN`:

In [12]:
import numpy as np
from scipy.stats import kurtosis, skew, median_abs_deviation
from scipy.fft import fft
from sklearn import metrics

fmn = np.mean(W, axis=-1)
print('FMN:', fmn.shape)

# Root of sum of squares
rss = np.sqrt(np.sum(W, axis=-1))
print('RSS:', rss.shape)

maxvalue = np.max(w, axis=-1)
print ('max:', maxvalue.shape)

minvalue = np.min(w, axis=-1)
print ('min:', maxvalue.shape)

mean = np.mean(w, axis=-1)
print ("mean: ", mean.shape)

std = np.std(w, axis=-1)
print ("std: ", std.shape)

kurt = kurtosis(w, axis=-1)
print ("Kurtosis: ", kurt.shape)

#skewn = skew(w, axis=-1)
#print ("Skewness: ", skewn.shape)

rms = np.sqrt(np.mean(w**2, axis=-1))
print ("RMS: ", rms.shape)

# auc = metrics.auc(w)
# print ("auc: ", auc.shape)

avm = np.mean(Wabs, axis=-1)
print("avm: ", avm.shape)

mad = median_abs_deviation(w, axis=-1)
print("mad: ", mad.shape)

mag = np.linalg.norm(w, axis=-1)
print("mag: ", mag.shape)

stdpsd = np.std(w, axis=-1)
print ("stdpsd: ", stdpsd.shape)

#fft = fft(w, axis=-1)
#print("fft: ", fft.shape)


FMN: (125, 13, 6)
RSS: (125, 13, 6)
max: (125, 13, 6)
min: (125, 13, 6)
mean:  (125, 13, 6)
std:  (125, 13, 6)
Kurtosis:  (125, 13, 6)
RMS:  (125, 13, 6)
avm:  (125, 13, 6)
mad:  (125, 13, 6)
mag:  (125, 13, 6)
stdpsd:  (125, 13, 6)


Após a aplicação de algumas características, juntamos todas elas no mesmo conjunto de dados e transformamos cada eletrodo em uma característica. Em outras palavras, o *shape* final que ficou no seguinte formato:`(125, 13, 17)`. Agora deverá ficar `(125 * 17, 13) => (2125, 13)`.

Se mais características fossem adicionadas, elas entrariam como multiplicação nas colunas. No exemplo anterior temos apenas uma característica desenvolvida. Se adicionarmos 4 características, o `shape` do vetor de características ficaria no seguinte formato: `(2125, 13 * 4) => (2125, 52)`. Explicando os dados, seriam 2125 amostras e 52 características.

In [13]:
# realização das transformações finais (TAREFA)

# finalizando o exemplo com a junção das duas características criadas
features = list()
for feature in (fmn, rss, mean, maxvalue, minvalue, std, kurt, rms, avm, mad, mag, stdpsd):
    feature = feature.transpose(0, 2, 1)
    feature = feature.reshape(feature.shape[0] * feature.shape[1],
                              feature.shape[2])
    features.append(feature)

# vetor de características final
X = np.concatenate(features, axis=-1)
print('Shape dos dados:', X.shape)

Shape dos dados: (750, 156)


### Adaptação do vetor de *labels*

Temos que adaptar o vetor de *labels* para ficar do mesmo tamanho (mesma quantidade de linhas) que o vetor de dados `X`

In [14]:
y = np.load('files/labels.npy')
print('Shape original dos labels', y.shape)

size = int(X.shape[0] / y.shape[0])
y = np.concatenate([y for i in range(size)])
print('Shape final dos labels', y.shape)

Shape original dos labels (125,)
Shape final dos labels (750,)


## Questões de projeto

1) Nem sempre os canais são vistos como características. Uma outra forma é adicionar os canais às amostras (reduzindo a quantidade de características e aumentando a quantidade de amostras). O resultado disso deve ser avaliado.

2) É comum a aplicação de algum algoritmo para reduzir todos os canais ou transformar apenas em um (que é o caso de aplicar a média de todos os eletrodos/canais).

3) Adicionar características ruins confundem o resultado? Características que não estão relacionadas ao domínio do problema pode ser ruim? Isso deve ser avaliado...

## Respostas 
1) Avaliado.

2) Sim, como demonstrado acima, foi adicionado todos os vetores de características em uma lista única, referente as extrações da região que está sendo analisada. Essa técnica é utilizada para reduzir e simplificar a análise do classificador.

3) Sim, pois como no experimento, estamos analisando a região parietal, para-occiptal, occipital, referente a visão do indivíduo. Caso for aplicado um cálculo de média em regiões que não processam informações da visão, como o lobo-frontal que é responsável pelas ações e movimentos, seria considerado como um ruído nos resultados do experimento.